In [ ]:
import os
import re
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

# from langchain.chat_models import ChatOpenAI


# Load API key from .env
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Initialize Groq LLM
llm = ChatGroq(model="llama3-8b-8192", temperature=0.2)



#llm = ChatOpenAI(
   # temperature=0.2,
   # model_name="gpt-3.5-turbo",  # or gpt-4 if available
   # openai_api_key=OPENAI_API_KEY
#)

# ✅ Tax Calculator Tool (Supports New & Old Regime)
def tax_calculator(input_str: str) -> str:
    try:
        match = re.search(r"(\d+[\d,]*)", input_str)
        if not match:
            return "❗Please provide a valid income amount."
        income = float(match.group(1).replace(",", ""))
        regime = "new" if "new" in input_str.lower() else "old"

        if regime == "new":
            tax = 0
            if income <= 300000:
                tax = 0
            elif income <= 600000:
                tax = 0.05 * (income - 300000)
            elif income <= 900000:
                tax = 15000 + 0.1 * (income - 600000)
            elif income <= 1200000:
                tax = 45000 + 0.15 * (income - 900000)
            elif income <= 1500000:
                tax = 90000 + 0.2 * (income - 1200000)
            else:
                tax = 150000 + 0.3 * (income - 1500000)

            return f"""✅ **Tax Calculation (New Regime)**  
Income: ₹{income:,.0f}  
Tax Payable: ₹{tax:,.0f}  
Slabs: ₹0–3L: 0%, ₹3L–6L: 5%, ₹6L–9L: 10%, ₹9L–12L: 15%, ₹12L–15L: 20%, >₹15L: 30%"""

        else:
            # Old Regime Slabs (FY 2023-24)
            tax = 0
            if income <= 250000:
                tax = 0
            elif income <= 500000:
                tax = 0.05 * (income - 250000)
            elif income <= 1000000:
                tax = 12500 + 0.2 * (income - 500000)
            else:
                tax = 112500 + 0.3 * (income - 1000000)

            # Rebate under section 87A if income <= ₹5L
            if income <= 500000:
                tax = 0

            return f"""✅ **Tax Calculation (Old Regime)**  
Income: ₹{income:,.0f}  
Tax Payable: ₹{tax:,.0f}  
Slabs: ₹0–2.5L: 0%, ₹2.5L–5L: 5%, ₹5L–10L: 20%, >₹10L: 30%  
Rebate under Section 87A (if income ≤ ₹5L) applied."""

    except Exception as e:
        return f"❗Error in tax calculation: {e}"

# ✅ Wikipedia Tool
wiki_api = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = Tool(
    name="wikipedia_search",
    func=WikipediaQueryRun(api_wrapper=wiki_api).run,
    description="Use this tool to answer general questions about Indian tax laws, deductions, or tax rules from Wikipedia."
)

# ✅ Tax Calculator Tool wrapper
# this function we have create for binding with LLM 
tax_tool = Tool(
    name="tax_calculator",
    func=tax_calculator,
    description="""Use this tool to calculate Indian income tax based on user's income and selected regime (new or old).
The user will mention the income in ₹ (like 8,00,000) and regime (new/old). Calculate using appropriate slabs.
"""
)

# ✅ Define all tools
tools = [tax_tool, wiki_tool]

# ✅ Agent Initialization
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)



# thsi is all BE coding


In [ ]:
result = agent.run("what are the verious tax labs in india for salaried ")




> Entering new AgentExecutor chain...
Thought: I need to provide information about the various tax slabs in India for salaried individuals. I can use the tax_calculator tool to get the tax slabs for the new and old regimes.

Action: tax_calculator
Action Input: "new" (assuming the user wants to know the tax slabs for the new regime)

Observation: ❗Please provide a valid income amount.
Thought:Action: tax_calculator
Action Input: "8,00,000" (assuming the user wants to know the tax slabs for an income of ₹8,00,000)

Observation: ✅ **Tax Calculation (Old Regime)**  
Income: ₹800,000  
Tax Payable: ₹72,500  
Slabs: ₹0–2.5L: 0%, ₹2.5L–5L: 5%, ₹5L–10L: 20%, >₹10L: 30%  
Rebate under Section 87A (if income ≤ ₹5L) applied.
Thought:Here's the continuation:

Action: wikipedia_search
Action Input: "What are the tax slabs in India for salaried individuals?"
Observation: Page: Taxation in India
Summary: Taxes in India are levied by the Central Government and the State Governments by virtue of pow

In [16]:
print(result)

The tax slabs in India for salaried individuals are as follows: ₹0–2.5L: 0%, ₹2.5L–5L: 5%, ₹5L–10L: 20%, >₹10L: 30%. Additionally, there is a rebate under Section 87A if the income is ≤ ₹5L.
